In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm

df = pd.read_csv('books.csv')
df.head()

,title,rating,price,stock,book_link,img_link
0,A Light in the Attic,Three,£51.77,In stock,https://books.toscrape.com/catalogue/a-light-i...,https://books.toscrape.com/../media/cache/2c/d...
1,Tipping the Velvet,One,£53.74,In stock,https://books.toscrape.com/catalogue/tipping-t...,https://books.toscrape.com/../media/cache/26/0...
2,Soumission,One,£50.10,In stock,https://books.toscrape.com/catalogue/soumissio...,https://books.toscrape.com/../media/cache/3e/e...
3,Sharp Objects,Four,£47.82,In stock,https://books.toscrape.com/catalogue/sharp-obj...,https://books.toscrape.com/../media/cache/32/5...
4,Sapiens: A Brief History of Humankind,Five,£54.23,In stock,https://books.toscrape.com/catalogue/sapiens-a...,https://books.toscrape.com/../media/cache/be/a...


## Approach-1

In [57]:
# First we will try to scrap the data of individual book and then automate the process for each and every book.
data = []

for link in tqdm(df['book_link']):
    res = requests.get(link)
    soup = BeautifulSoup(res.text, 'html.parser')
    typ = soup.find('ul', class_='breadcrumb').find_all('a')[2].text    # Something new using find on find, [2] because we need
                                                                    # to know the category of it.
        
    upc           =  soup.find('table', class_='table table-striped').find_all('td')[0].text
    price_x       = soup.find('table', class_='table table-striped').find_all('td')[2].text[2:]
    price_i       = soup.find('table', class_='table table-striped').find_all('td')[3].text[2:]
    tax           = soup.find('table', class_='table table-striped').find_all('td')[4].text[2:]
    qn         = soup.find('table', class_='table table-striped').find_all('td')[5].text
    reviews       = soup.find('table', class_='table table-striped').find_all('td')[6].text

    data.append([typ, tax, price_x,price_i, upc, qn,reviews])

    

 33%|█████████████████████████▉                                                     | 328/1000 [05:03<10:21,  1.08it/s]


ConnectionError: HTTPSConnectionPool(host='books.toscrape.com', port=443): Max retries exceeded with url: /catalogue/carry-on-warrior-thoughts-on-life-unarmed_672/index.html (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x0000016F14AE5910>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))

## Approach-2

In [51]:
data = []

for link in tqdm(df['book_link']):
    res = requests.get(link)
    soup = BeautifulSoup(res.text, 'html.parser')
    typ = soup.find('ul', class_='breadcrumb').find_all('a')[2].text    # Something new using find on find, [2] because we need
                                                                    # to know the category of it.
        
    temp = soup.find('table', class_='table table-striped').find_all('td')
    upc           = temp[0].text
    price_x       = temp[2].text[2:]
    price_i       = temp[3].text[2:]
    tax           = temp[4].text[2:]
    qn            = temp[5].text
    reviews       = temp[6].text

    data.append([tax, price_x,price_i, upc, qn,reviews])


 98%|█████████████████████████████████████████████████████████████████████████████  | 976/1000 [14:55<00:22,  1.09it/s]


ConnectionError: HTTPSConnectionPool(host='books.toscrape.com', port=443): Max retries exceeded with url: /catalogue/hawkeye-vol-1-my-life-as-a-weapon-hawkeye-1_24/index.html (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x0000016F14C1B550>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed'))

Task to do after this are:
1. Creating a dataframe out of the data.
2. Concatenating the dataframe created with the previous existing dataframe.
3. Making a csv file and saving the dataframes together.

In [56]:
data[0]
df = pd.DataFrame(data, columns=['category', 'price_e_tax', 'price_i_tax', 'quantity', 'upc', 'reviews'])
df.head()

df.to_csv('data.csv')

['0.00', '51.77', '51.77', 'a897fe39b1053632', 'In stock (22 available)', '0']

## Combining both the datasets

In [54]:
df_1 = pd.read_csv('books.csv')
# df_2 = pd.read_csv('data.csv')

In [55]:
df_1.head()
# df_2.head()


,title,rating,price,stock,book_link,img_link
0,A Light in the Attic,Three,£51.77,In stock,https://books.toscrape.com/catalogue/a-light-i...,https://books.toscrape.com/../media/cache/2c/d...
1,Tipping the Velvet,One,£53.74,In stock,https://books.toscrape.com/catalogue/tipping-t...,https://books.toscrape.com/../media/cache/26/0...
2,Soumission,One,£50.10,In stock,https://books.toscrape.com/catalogue/soumissio...,https://books.toscrape.com/../media/cache/3e/e...
3,Sharp Objects,Four,£47.82,In stock,https://books.toscrape.com/catalogue/sharp-obj...,https://books.toscrape.com/../media/cache/32/5...
4,Sapiens: A Brief History of Humankind,Five,£54.23,In stock,https://books.toscrape.com/catalogue/sapiens-a...,https://books.toscrape.com/../media/cache/be/a...


In [ ]:
# Making individual dataframe elements from the separate dataframes.

df['title'] = df_1['title']
df['upc'] = df_2['upc']
df['category'] = df_2['category']
df['price_e_tax'] = df_2['price_e_tax']
df['price_i_tax'] = df_2['price_i_tax']
df['tax'] = df_2['tax']
df['rating'] = df_2['rating']
df['reviews'] = df_2['reviews']
df['quantity'] = df_2['quantity']
df['stock'] = df_1['stock']
df['book_link'] = df_1['book_link']
df['img_link'] = df_1['img_link']



In [2]:
link = 'https://www.wikipedia.org/'

res = requests.get(link)

In [4]:
soup = BeautifulSoup(res.text, 'html.parser')

In [5]:
print(soup.find())

<html class="no-js" lang="en">
<head>
<meta charset="utf-8"/>
<title>Wikipedia</title>
<meta content="Wikipedia is a free online encyclopedia, created and edited by volunteers around the world and hosted by the Wikimedia Foundation." name="description"/>
<script>
document.documentElement.className = document.documentElement.className.replace( /(^|\s)no-js(\s|$)/, "$1js-enabled$2" );
</script>
<meta content="initial-scale=1,user-scalable=yes" name="viewport"/>
<link href="/static/apple-touch/wikipedia.png" rel="apple-touch-icon"/>
<link href="/static/favicon/wikipedia.ico" rel="shortcut icon"/>
<link href="//creativecommons.org/licenses/by-sa/3.0/" rel="license"/>
<style>
.sprite{background-image:linear-gradient(transparent,transparent),url(portal/wikipedia.org/assets/img/sprite-e99844f6.svg);background-repeat:no-repeat;display:inline-block;vertical-align:middle}.svg-Commons-logo_sister{background-position:0 0;width:47px;height:47px}.svg-MediaWiki-logo_sister{background-position:0 -47px